<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">


# <center> Проект: Анализ резюме из HeadHunter


In [ ]:
import pandas as pd
import numpy as np
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots

# Исследование структуры данных


1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла.


In [ ]:
hh_data_original = pd.read_csv("../tables/dst-3.0_16_1_hh_database.csv", sep=";")
hh_data = hh_data_original.copy()  # copy dataframe to protect original table

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.


In [ ]:
hh_data.head()

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.


In [ ]:
hh_data.info()

4. Обратите внимание на информацию о числе непустых значений.


In [ ]:
hh_data.notna().sum()

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
hh_data.describe(include=object)

In [ ]:
# block with code for answer questions:
hh_data["Опыт работы"].describe(include=object())
hh_data["Последняя/нынешняя должность"].describe(include=object())

# Преобразование данных


1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:

- Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
- Неоконченное высшее образование 2000 Балтийская государственная академия рыбопромыслового флота…
  Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

_Совет: проверяйте полученные категории, например, с помощью метода unique()_


In [ ]:
def find_education(full_educ_row: str) -> str:
    """
    Return education level string.
    """
    pattern = r"(.+)(\d{4})" # initialize patter for search
    match = re.search(pattern, full_educ_row)
    all_symbols_before_year = match.group(1) if match else ""
    return all_symbols_before_year[:-1]


def create_education(education_level: str) -> str:
    """
    Execute education from eduction level string.
    """
    education = education_level.lower().split()
    if education[0] == "неоконченное" or education[1] == "специальное":
        return str(education[0] + " " + education[1])
    else:
        return education[0]

In [ ]:
hh_data['Образование'] = hh_data['Образование и ВУЗ'].apply(lambda x: create_education(find_education(x)))

# check if we have only four required values in new column
assert set(hh_data['Образование'].unique()) == set(['неоконченное высшее', 'высшее', 'среднее специальное', 'среднее'])

In [ ]:
# block with code for answer questions:
len(hh_data[hh_data["Образование"] == "среднее"])

# drop column 'Образование и ВУЗ'
hh_data.drop(columns="Образование и ВУЗ", axis=1, inplace=True)

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:

- Мужчина , 39 лет , родился 27 ноября 1979
- Женщина , 21 год , родилась 13 января 2000
  Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:

- Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина.
- Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

_Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '.
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим._


In [ ]:
def find_user_data(row: str) -> tuple:
    """
    Return tuple with gender and age.
    """
    user_data = row.split(' , ')
    return user_data[0][0], user_data[1].split()[0]

In [ ]:
hh_data[["Пол", "Возраст"]] = (
    hh_data["Пол, возраст"].apply(find_user_data).apply(pd.Series)
)
hh_data["Возраст"] = hh_data["Возраст"].astype(pd.Int64Dtype())
# use assert to check if i did correct converts for new columns
assert set(hh_data["Пол"].unique()) == set(["М", "Ж"])
assert hh_data["Возраст"].dtype == "Int64"

In [ ]:
# block of code to answer questions
round((len(hh_data[hh_data["Пол"] == "Ж"]) / len(hh_data["Пол"])) * 100, 2)
round(hh_data["Возраст"].mean(), 2)

# drop column 'Пол, возраст'
hh_data.drop(columns=["Пол, возраст"], axis=1, inplace=True)

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**.

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:

- Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
- Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
- В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
- В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
  - Опыт работы 3 года 2 месяца…
  - Опыт работы 4 года…
  - Опыт работы 11 месяцев…
  - Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
def find_experience(value) -> int:
    """
    Return work experience in months.
    """
    if value is np.nan:
        return np.nan
    elif value == "Не указано":
        return np.nan
    experience = value[0]
    i = 1
    # try to find all symbols up to the next
    # capital letter, which is at the beginning of the
    # part about the period of work
    while (not value[i].isupper()) and i < len(value)-1:
        experience += value[i]
        i += 1

    month_pattern = re.compile(r"(\d+)\s+(месяц?)")
    match_month = re.search(month_pattern, experience)
    month = int(match_month.group(1)) if match_month else 0

    year_pattern = re.compile(r"(\d+)\s+(год|лет?)")
    match_year = re.search(year_pattern, experience)
    year = int(match_year.group(1)) if match_year else 0

    return year*12 + month

In [ ]:
hh_data["Опыт работы (месяц)"] = hh_data["Опыт работы"].apply(find_experience)
hh_data["Опыт работы (месяц)"] = hh_data["Опыт работы (месяц)"].astype(
    pd.Int64Dtype()
)  # experience should be int number

# check if we execute work experience in months correctly
assert hh_data["Опыт работы (месяц)"].isna().sum() != 0  # NaN should be presented
assert hh_data["Опыт работы (месяц)"].dtype == "Int64"

In [ ]:
# block to answer questions
hh_data["Опыт работы (месяц)"].median()

# drop column Опыт работы
hh_data.drop(columns="Опыт работы", axis=1, inplace=True)

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

- Москва , не готов к переезду , готов к командировкам
- Москва , м. Беломорская , не готов к переезду, не готов к командировкам
- Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

- Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

  Список городов-миллионников:

  <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
  </code>
  Инфорация о метро, рядом с которым проживает соискатель нас не интересует.

- Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
  - … , готов к переезду , …
  - … , не готова к переезду , …
  - … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
  - … , хочу переехать (США) , …
    Нас интересует только сам факт возможности или желания переезда.
- Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
  - … , готов к командировкам , …
  - … , готова к редким командировкам , …
  - … , не готов к командировкам , …
    Нас интересует только сам факт готовности к командировке.
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.

Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

_Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе._


In [ ]:
def find_city(row: str) -> str:
    """
    Return city or "город-миллионник", if it represented in list, otherwise 'другое'.
    """
    capital_cities = [
        "Москва",
        "Санкт-Петербург",
    ]
    million_cities = [
        "Новосибирск",
        "Екатеринбург",
        "Нижний Новгород",
        "Казань",
        "Челябинск",
        "Омск",
        "Самара",
        "Ростов-на-Дону",
        "Уфа",
        "Красноярск",
        "Пермь",
        "Воронеж",
        "Волгоград",
    ]
    city = row.split()[0]
    if city in capital_cities:
        return city
    elif city in million_cities:
        return "город-миллионник"
    else:
        return "другие"


def is_ready(row: str, ready_for="") -> bool:
    """
    Return if 'ready_for' string  is presented in row.
    """
    for elem in row.split(","):
        if ready_for in elem:
            return "не" not in elem.split("(")[0]
    return False  # if ready_for is not represented in row

In [ ]:
hh_data["Город"] = hh_data["Город, переезд, командировки"].apply(find_city)
hh_data["Готовность к переезду"] = hh_data["Город, переезд, командировки"].apply(
    lambda x: is_ready(x, "перее")
)
hh_data["Готовность к командировкам"] = hh_data["Город, переезд, командировки"].apply(
    lambda x: is_ready(x, "ком")
)

# check correct values for each new column
possible_cities = [
    "Москва",
    "Санкт-Петербург",
    "город-миллионник",
    "другие"
]
for city in hh_data["Город"]:
    assert city in possible_cities
assert set(hh_data["Готовность к переезду"].unique()) == set([True, False])
assert set(hh_data["Готовность к командировкам"].unique()
           ) == set([True, False])

In [ ]:
# block to answer questions
petersburg = round(len(hh_data[hh_data["Город"] == "Санкт-Петербург"]) / len(hh_data["Город"]) * 100)
all_ready_to_move = round(len(
    hh_data[
        (hh_data["Готовность к переезду"] == True)
        & (hh_data["Готовность к командировкам"] == True)
    ]
) / hh_data.shape[0] * 100)
print(f"{petersburg} процентов соискателей живут в Санкт-Петербурге")
print(f"{all_ready_to_move} процентов соискателей готовы одновременно и к переездам, и к командировкам")

# drop column 'Город, переезд, командировки'
hh_data.drop(columns="Город, переезд, командировки", axis=1, inplace=True)

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
   На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:

- полная занятость, частичная занятость
- частичная занятость, проектная работа, волонтерство
- полный день, удаленная работа
- вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы


In [ ]:
def one_hot_encoding(pd, columns):
    """
    Performs one hot encoding on chosen columns.
    """
    for column in columns:
        list_of_values = list(
            set(
                [
                    category.strip()
                    for work in pd[column]
                    for category in work.split(",")
                ]
            )
        )

        for type in list_of_values:
            pd[type] = False
        for type in list_of_values:
            pd[type] = pd[column].str.contains(
                type
            )  # will use contains to improve speed of searching

    return pd

In [ ]:
hh_data = one_hot_encoding(hh_data, columns=["Занятость", "График"])

In [ ]:
# block to answer questions
len(hh_data[(hh_data["проектная работа"] == True)
    & hh_data["волонтерство"] == True])
len(hh_data[(hh_data["гибкий график"] == True)
    & hh_data["вахтовый метод"] == True])

# drop columns 'Занятость', 'График'
hh_data.drop(columns=["Занятость", "График"], axis=1, inplace=True)

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**.
   В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:

- 30000 руб.
- 50000 грн.
- 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:

- "currency" - наименование валюты в ISO кодировке,
- "date" - дата,
- "proportion" - пропорция,
- "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).

Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>

Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.


Итак, давайте обсудим возможный алгоритм преобразования:

1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# read table with currencies
currency_data = pd.read_csv("../tables/ExchangeRates.csv", sep=",")
currencies = currency_data.copy()
currencies.head()

In [ ]:
def find_date(datetime_str, date_format):
    "Return converted time in selected format."
    return pd.to_datetime(datetime_str, format=date_format).date().strftime("%Y-%m-%d")

In [ ]:
hh_data["Обновление резюме"] = hh_data["Обновление резюме"].apply(
    lambda date: find_date(date, "%d.%m.%Y %H:%M")
)
currencies["date"] = currencies["date"].apply(
    lambda date: find_date(date, "%d/%m/%y"))

In [ ]:
def convert_to_iso_currecny(currency):
    """
    Return to ISO currency, if it is not in this format
    """
    not_iso_currecnies = {"бел.руб.": "BYN", "сум": "UZS", "грн.": "UAH"}
    if currency in not_iso_currecnies:
        return not_iso_currecnies[currency]
    return currency

In [ ]:
hh_data["ЗП (руб)"] = 0.0

for index, row in hh_data.iterrows():
    wage, currency = row["ЗП"].split()
    wage = float(wage)
    if currency == "руб.":
        hh_data.at[index, "ЗП (руб)"] = wage
    else:
        currency = convert_to_iso_currecny(currency)
        mask = currencies[
            (currencies["date"] == row["Обновление резюме"])
            & (currencies["currency"] == currency)
        ]
        new_wage = mask["close"].iloc[0] / mask["proportion"].iloc[0] * wage
        hh_data.at[index, "ЗП (руб)"] = new_wage

In [ ]:
# block to answer questions
hh_data["ЗП (руб)"].describe()

# drop column 'ЗП'
hh_data.drop(columns="ЗП", axis=1, inplace=True)

In [ ]:
# check if all editions were made correctly
assert hh_data.shape[1] == 23
assert hh_data.shape[0] == 44744

# Исследование зависимостей в данных


1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
   _Совет: постройте гистограмму и коробчатую диаграмму рядом._


In [ ]:
figure_basic_size = (12, 4)

In [ ]:

fig1 = px.histogram(hh_data, x='Возраст', nbins=20, title='Распределение возраста')
fig2 = px.box(hh_data, x='Возраст', title='Коробчатая диаграмма возраста')

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['Распределение возраста', 'Коробчатая диаграмма возраста'])

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.update_layout(showlegend=False, title_text="Возраст")

display(fig)
fig.write_html("../graphics/age_histagram_box.html")

In [ ]:
mode = np.argmax(np.bincount(hh_data['Возраст']))
print(f"Мода распределения: {mode}")

min_age = np.min(hh_data['Возраст'])
max_age = np.max(hh_data['Возраст'])
print(f"Минимальный возраст: {min_age}", f", встречается в данных {(hh_data['Возраст'] <= min_age).sum()} раз")
print(f"Максимальный возраст: {max_age}", f", встречается в данных {(hh_data['Возраст'] >= max_age).sum()} раз")

# Find anomaly values:
std_dev = np.std(hh_data['Возраст'])
anomalies = [value for value in hh_data['Возраст'] if abs(value - np.mean(hh_data['Возраст'])) > 5 * std_dev]
print(f"Аномалии: {anomalies}")

По графикам и полученным расчетам, мы можем сделать вывод, что большая часть соискателей находится в возрасте 30 лет.
Есть соискатели возрастом 14 и 100 - для наших данных это, возможно, ошибка заполнения анкеты.
Также мы видим, что есть соискатели 65 лет и старше.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
   _Совет: постройте гистограмму и коробчатую диаграмму рядом._


In [ ]:
# create mask to avoid NaN values
mask = ~hh_data['Опыт работы (месяц)'].isna()

fig1 = px.histogram(hh_data[mask], x='Опыт работы (месяц)', nbins=20, title='Распределение опыта работы')
fig2 = px.box(hh_data[mask], x='Опыт работы (месяц)', title='Коробчатая диаграмма опыта работы')
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['Распределение опыта работы', 'Коробчатая диаграмма опыта работы'])

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(showlegend=False, title_text="Опыт работы (месяц)")

display(fig)
fig.write_html('../graphics/work_experience_deviation.html')

In [ ]:
# additional block to answer questions and make additional decisions
mode = np.nanmax(np.bincount(hh_data['Опыт работы (месяц)'][mask].astype(int)))
print(f"Мода распределения: {mode}")

min_experience = np.nanmin(hh_data['Опыт работы (месяц)'][mask])
min_count = (hh_data['Опыт работы (месяц)'][mask] <= min_experience).sum()
max_experience = np.nanmax(hh_data['Опыт работы (месяц)'][mask])
max_count = (hh_data['Опыт работы (месяц)'][mask] >= max_experience).sum()
print(f"Минимальный опыт работы: {min_experience} месяцев", f"встречается {min_count} раз")
print(f"Максимальный опыт работы: {max_experience} месяцев", f"встречается {max_count} раз")

std_dev = np.nanstd(hh_data['Опыт работы (месяц)'][mask])
# attention! Work with anomalies can take more than 1 minute!
anomalies = [value for value in hh_data['Опыт работы (месяц)'][mask] if abs(value - np.nanmean(hh_data['Опыт работы (месяц)'][mask])) > 6 * std_dev]
print(f"Аномалии: {anomalies}")

При анализе графиков, можно заметить следующие
- Наиболее количество резюме приходятся на диапазоне от 50 месяцев (4х лет)  до 149 месяцев (12 лет) работы.
- В выборке есть аномалия - опыт работы 1188 месяцев (99 лет) - что больше жизни человека, скорее всего ошибка в данных
- После 149х месяцев(12 лет) опыта работы количество соискателкй начинает снижаться


3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
   _Совет: постройте гистограмму и коробчатую диаграмму рядом._


In [ ]:
fig1 = px.histogram(hh_data, x='ЗП (руб)', nbins=20,
                    title='Распределение заработной платы')
fig2 = px.box(hh_data, x='ЗП (руб)', color_discrete_sequence=[
              'blue'], title='Коробчатая диаграмма заработной платы')
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=[
                       'Распределение заработной платы', 'Коробчатая диаграмма заработной платы'])

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(showlegend=False, title_text="Заработная плата")

display(fig)
fig.write_html('../graphics/salary_deviation.html')

In [ ]:
# additional block to answer questions and make additional decisions
mode_salary = np.argmax(np.bincount(hh_data['ЗП (руб)']))
print(f"Мода распределения: {mode_salary} руб")

min_salary = np.min(hh_data['ЗП (руб)'])
max_salary = np.max(hh_data['ЗП (руб)'])
print(f"Минимальная заработная плата: {min_salary} руб")
print(f"Максимальная заработная плата: {max_salary} руб")

std_dev_salary = np.std(hh_data['ЗП (руб)'])
anomalies_salary = [value for value in hh_data['ЗП (руб)'] if abs(value - np.mean(hh_data['ЗП (руб)'])) > 3 * std_dev_salary]
print(f"Аномалии: {anomalies_salary}")

more_million = (hh_data['ЗП (руб)'] > 1000000).sum()
print(f"Количество резюме с требованием ЗП больше 1000000: {more_million}")
less_thousand = (hh_data['ЗП (руб)'] < 1000).sum()
print(f"Количество резюме с требованием ЗП меньше 1000: {less_thousand}")

In [ ]:
# additional graph for salary research, anomalies make it difficult to read first graph
mask = hh_data['ЗП (руб)'] < 1000000

fig1 = px.histogram(hh_data[mask], x='ЗП (руб)', nbins=20,
                    title='Распределение заработной платы', color_discrete_sequence=['green'])
fig2 = px.box(hh_data[mask], x='ЗП (руб)', color_discrete_sequence=[
              'green'], title='Коробчатая диаграмма заработной платы')
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=[
                       'Распределение заработной платы', 'Коробчатая диаграмма заработной платы'])

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(showlegend=False,
                  title_text="Заработная плата (менее 1 млн. руб.)")

display(fig)
fig.write_html('../graphics/salary_additional_deviation.html')

По итогу анализа четырех графиков (распределение заработной платы и заработная плата (менее 1 млн руб.)) можно заметить следующее:
- В данных присутствуют выбросы. Так, пять резюме содержат заработную плату более 1 млн рублей, а 84 резюме - заработную плату меньше 1000 рублей.
- В данных присутствует заработная плата равная 24304876 рублей, что может быть ошибкой.
- Большая часть резюме сосредоточена в размере заработной платы до 50000 рублей. На дополнительном зеленом графике видно, что 50% резюме сосредоточено в диапазоне до 50000 рублей.
- График ассиметричен вправо

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
   _Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?_


In [ ]:
# attention! mask was created in previous Python block above
# mask = hh_data['ЗП (руб)'] < 1000000
fig1 = px.box(hh_data[mask], x='Образование', y='ЗП (руб)', category_orders={
              'Образование': hh_data['Образование'].unique()}, title='Коробчатая диаграмма заработной платы')

fig2 = px.scatter(hh_data[mask], x='Образование', y='ЗП (руб)', category_orders={
                  'Образование': hh_data['Образование'].unique()}, title='Точечный график заработной платы')

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=[
                       'Коробчатая диаграмма заработной платы', 'Точечный график заработной платы'])

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(showlegend=False,
                  title_text="Заработная плата по уровню образования")

display(fig)
fig.write_html("../graphics/salary_by_education_deviation_.html")


На графике видно, что с повышением уровня образования заработная плата в среднем увеличивается. Медианная заработная плата для высшего образования составляет 100 000 рублей, для неоконченного высшего образования - 70 000 рублей, для среднего специального образования - 60 000 рублей, для среднего образования - 50 000 рублей.
Для прогнозирования заработной платы важно учитывать уровени образования.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
   _Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?_


In [ ]:
# attention! mask was created in previous Python block above
# mask = hh_data['ЗП (руб)'] < 1000000
fig = px.box(hh_data[mask], x='Город', y='ЗП (руб)', category_orders={
             'Город': hh_data[mask]['Город'].unique()}, title='Коробчатая диаграмма заработной платы по городам')
fig.update_layout(showlegend=False, title_text="Заработная плата по городам")
display(fig)

fig.write_html("../graphics/salary_by_city_deviation.html")

На графике распределения заработной платы по городам видно, что в крупных городах, таких как Москва и Санкт-Петербург, желаемая заработная плата выше, чем в других городах. Медианная заработная плата в Москве составляет 85 000 рублей, в Санкт-Петербурге - 60 000 рублей, в городах-миллионниках - 40 000 рублей, в других городах - 40 000 рублей. На графике также видно, что в Москве заработная плата выше, чем в Санкт-Петербурге. Это может быть связано с тем, что Москва является столицей России и центр экономической активности.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.


In [ ]:
# create stand-alone df to create figure
median_salary_df = pd.DataFrame(index=pd.MultiIndex.from_product(
    [['Готовность к переезду', 'Готовность к командировкам'], ['False', 'True']], names=['Признак', 'Значение']))

for move_value in ['False', 'True']:
    for travel_value in ['False', 'True']:
        mask = (hh_data['Готовность к переезду'] == (move_value == 'True')) & (
            hh_data['Готовность к командировкам'] == (travel_value == 'True'))
        median_salary = hh_data[mask]['ЗП (руб)'].median()
        median_salary_df.loc[('Готовность к переезду',
                              move_value), 'Медиана'] = median_salary
        median_salary_df.loc[('Готовность к командировкам',
                              travel_value), 'Медиана'] = median_salary

fig = go.Figure()
fig.add_trace(go.Bar(x=median_salary_df.index.get_level_values('Признак') + ': ' + median_salary_df.index.get_level_values('Значение').astype(str),
                     y=median_salary_df['Медиана'], marker_color='lightblue'))
fig.update_layout(title='Медианная заработная плата в зависимости от готовности к переезду и командировкам',
                  xaxis_title='Готовность (True/False)', yaxis_title='Медианная заработная плата (руб)')

display(fig)
fig.write_html("../graphics/median_salary_combined_final.html")

Желаемая заработная плата соискателей зависит от их готовности к переезду и командировкам. Соискатели, которые готовы к командировкам и переезду, в среднем просят более высокую заработную плату, чем соискатели, которые не готовы.  
Соискатели, которые готовы к командировкам, в среднем просят более высокую заработную плату. На графике видно, что медианное значение заработной платы для соискателей, которые готовы к командировкам и/или переезду, составляет 67 тыс. руб. Для соискателей, которые не готовы к переезду/комадировкам соотвественно: 50 тыс. руб. и 60 тыс. руб. 

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.


In [ ]:
# attention! mask was created in previous Python block above
# mask = hh_data['ЗП (руб)'] < 1000000
pivot_table = pd.pivot_table(hh_data[mask], values='ЗП (руб)',
                             index='Возраст', columns='Образование', aggfunc=np.median)

fig = make_subplots(rows=1, cols=1,
                    subplot_titles=['Тепловая карта медианной заработной платы по возрасту и образованию'])
heatmap = go.Heatmap(z=pivot_table.values,
                     x=pivot_table.columns,
                     y=pivot_table.index,
                     colorscale='Viridis',
                     zmin=0,
                     zmax=pivot_table.max().max(),
                     colorbar=dict(title='Медианная заработная плата (руб)'))
fig.add_trace(heatmap)
fig.update_layout(title_text='Тепловая карта медианной заработной платы по возрасту и образованию',
                  xaxis=dict(title='Образование'),
                  yaxis=dict(title='Возраст'))

display(fig)
fig.write_html("../graphics/salary_heatmap.html")

Средняя заработная плата соискателей увеличивается с возрастом. На графике видно, что медианное значение заработной платы для соискателей в возрасте от 20 до 30 лет составляет 55 000 рублей, для соискателей в возрасте от 30 до 40 лет - 80 000 рублей.  
Соискатели с высшим образованием в среднем просят более высокую заработную плату, чем соискатели с неполным высшим образованием или средним образованием. На графике видно, что медианное значение заработной платы для соискателей с высшим образованием составляет 100 000 рублей, для соискателей с неполным высшим образованием - 80 000 рублей, а для соискателей со средним образованием - 50 000 рублей.  

Таким образом, желаемая заработная плата соискателей зависит от их возраста и уровня образования.

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)


In [ ]:
mask = (~hh_data['Возраст'].isna()) & (~hh_data['Опыт работы (месяц)'].isna())

filtered_data = hh_data[mask]
filtered_data['Опыт работы (год)'] = filtered_data['Опыт работы (месяц)'] / 12

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=filtered_data['Возраст'], y=filtered_data['Опыт работы (год)'], mode='markers', name='Соискатели'))
fig.add_trace(go.Scatter(x=[0, 100], y=[0, 100], mode='lines', line=dict(
    color='red', dash='dash'), name='Опыт работы = Возраст'))
anomalies = filtered_data[filtered_data['Опыт работы (год)']
                          >= filtered_data['Возраст']]
fig.add_trace(go.Scatter(x=anomalies['Возраст'], y=anomalies['Опыт работы (год)'],
              mode='markers', marker=dict(color='orange'), name='Аномалии'))
fig.update_layout(title='Диаграмма рассеяния: Опыт работы от возраста',
                  xaxis_title='Возраст',
                  yaxis_title='Опыт работы (год)',
                  showlegend=True)

display(fig)
fig.write_html("../graphics/salary_by_experience_scatter.html")

На диаграмме видно, что в целом существует положительная корреляция между опытом работы и желаемой заработной платой. Это означает, что соискатели с большим опытом работы в среднем просят более высокую заработную плату, чем соискатели с меньшим опытом работы.

Однако на диаграмме также видно, что существуют аномалии, где возраст соискателя меньше, чем его опыт. На графике такие аномалии отображены желтыми точками. 

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
avg_salary_by_education_gender = hh_data.groupby(['Образование', 'Пол'])[
    'ЗП (руб)'].mean().reset_index()
fig = px.bar(avg_salary_by_education_gender,
             x='Образование',
             y='ЗП (руб)',
             color='Пол',
             labels={'ЗП (руб)': 'Средняя Заработная плата (руб)'},
             title='Средние зарплаты у мужчин и женщин по видам образования',
             category_orders={'Образование': sorted(hh_data['Образование'].unique())})
fig.update_layout(xaxis_title='Образование',
                  yaxis_title='Средняя Заработная плата (руб)')

display(fig)
fig.write_html("../graphics/salary_by_gender.html")

В целом, соискатели-мужчины просят более высокую заработную плату, чем соискатели-женщины. На графике видно, что вне зависимости от уровня образования, соискатели-мужчины получают с таким же уровенм образования примерно на 15 % больше.
На графике видно, что медианное значение заработной платы для соискателей-мужчин с высшим образованием составляет 88 000 рублей, а для соискателей-женщин с высшим образованием - 65 000 рублей.

In [ ]:
def is_auto_enabled(row):
    if 'Не' in row:
        return False
    return True

In [ ]:
# Add one-code-hot encoding to column 'Авто'
hh_data['Авто'] = hh_data['Авто'].apply(is_auto_enabled)
mask = hh_data['Готовность к переезду'] == True

fig = px.histogram(filtered_data,
                   x='Город',
                   color='Авто',
                   barmode='group',
                   category_orders={'Город': sorted(hh_data[mask]['Город'].unique())},
                   labels={'count': 'Количество соискателей'},
                   title='Готовность к переезду от наличия автомобиля с учетом города',
                   color_discrete_map={True: 'green', False: 'blue'})
fig.update_layout(xaxis_title='Город', yaxis_title='Количество соискателей', legend_title='Наличие автомобиля')
fig.update_xaxes(tickangle=45, tickmode='array')
display(fig)
fig.write_html("../graphics/ready_to_move_by_auto.html")


Анализируя график, можем сказать что соискатели не указавшие наличие автомоблия более склонны к переездам и наибольшее количество таких соискателей находится в Москве

# Очистка данных


1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их.


In [ ]:
initial_rows = hh_data.shape[0]
hh_data_no_duplicates = hh_data.drop_duplicates()
duplicates_removed = initial_rows - hh_data_no_duplicates.shape[0]
print(f"Количество удаленных полных дубликатов: {duplicates_removed}")
print("Размер таблицы", hh_data.shape[0])

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах.


In [ ]:
missing_values = hh_data.isnull().sum()
print("Число пропусков в каждом столбце:")
print(missing_values)
experience_missing = missing_values['Опыт работы (месяц)']
print(f"\nКоличество пропусков в столбце 'Опыт работы (месяц)': {experience_missing}")
print("Размер таблицы", hh_data.shape[0])

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.


In [ ]:
median_experience = hh_data['Опыт работы (месяц)'].median()
hh_data['Опыт работы (месяц)'].fillna(median_experience, inplace=True)
hh_data.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'], inplace=True)

# check if we have deleted all null values
assert (hh_data.isnull().sum() == 0).all()

resulting_mean_experience = round(hh_data['Опыт работы (месяц)'].mean())
print(f"Результирующее среднее значение в столбце 'Опыт работы (месяц)': {resulting_mean_experience}")

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.


In [ ]:
hh_data = hh_data[(hh_data['ЗП (руб)'] <= 1000000) & (hh_data['ЗП (руб)'] >= 1000)]
print("Число строк после удаления выбросов:", hh_data.shape[0])

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
hh_data = hh_data[(hh_data['Возраст']) > (hh_data['Опыт работы (месяц)'] // 12)]
hh_data.shape[0]

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?


In [ ]:
fig = plt.yscale('log')

histplot = sns.histplot(np.log1p(hh_data['Возраст']), bins=30, color='blue', alpha=0.7)

# use 3-sigmas method
log_mean_age = np.log1p(hh_data['Возраст']).mean()
log_std_age = np.log1p(hh_data['Возраст']).std()
lower_bound_log = log_mean_age - 3 * log_std_age
upper_bound_log = log_mean_age + 3 * log_std_age

histplot.axvline(log_mean_age, color='k', linestyle='dashed', linewidth=2, label='Среднее значение')

histplot.axvline(lower_bound_log, color='green', linestyle='dashed', linewidth=2, label='Нижняя граница (3 сигмы)')
histplot.axvline(upper_bound_log, color='green', linestyle='dashed', linewidth=2, label='Верхняя граница (4 сигмы)')

plt.legend()

plt.xlabel('log(Возраст)')
plt.ylabel('Частота (логарифмический масштаб)')
plt.title('Распределение возраста (логарифмический масштаб) с линиями для среднего и интервала 3 сигм')

plt.show()

In [ ]:
log_outliers = hh_data[(np.log1p(hh_data['Возраст']) > (log_mean_age + 3.8 * log_std_age))]

print("Выбросы с использованием метода z-отклонений (послабление на 4 сигмы вправо):")
print(log_outliers[['Возраст']])

In [ ]:
log_outliers = hh_data[(np.log1p(hh_data['Возраст']) > (log_mean_age + 4 * log_std_age))]
hh_data_filtered = hh_data.drop(log_outliers.index)
print(f"\nЧисло строк в данных после удаления выбросов: {hh_data_filtered.shape[0]}")

Анализируя график и рассчеты, можно сделать следующие заключения:
- График имеет асимметричен вправо
- За верхней границей трех-сигм (в нашем случае с послаблением +1 сигму) количество соискатлей резко сокращается. Возможно это вызвано тем, что соискателям столь преклонного возраста уже сложно работать.
